In [1]:
import numpy as np
import timeit
import random

In [2]:
from IO import Reader, Writer, Data, Sentence, Token
from feature import FeatureMapping
from model import Model

# Decoder (Eisner's Algorithm)

In [3]:
test_matrix = np.array([[-10000, 9, 10, 9], 
               [-10000, -10000, 20, 3], 
               [-10000, 30, -10000, 30],
               [-10000, 11, 0, -10000]])
test_matrix[0, 2]

10

In [4]:
# Some constants
L, R = 0, 1
I, C = 0, 1
DIRECTIONS = (L, R)
COMPLETENESS = (I, C)
NEG_INF = -float('inf')


class Span(object):
    def __init__(self, left_idx, right_idx, head_side, complete):
        self.data = (left_idx, right_idx, head_side, complete)

    @property
    def left_idx(self):
        return self.data[0]

    @property
    def right_idx(self):
        return self.data[1]

    @property
    def head_side(self):
        return self.data[2]

    @property
    def complete(self):
        return self.data[3]

    def __str__(self):
        return "({}, {}, {}, {})".format(
            self.left_idx,
            self.right_idx,
            "L" if self.head_side == L else "R",
            "C" if self.complete == C else "I",
        )

    def __repr__(self):
        return self.__str__()

    def __hash__(self):
        return hash(self.data)

    def __eq__(self, other):
        return isinstance(other, Span) and hash(other) == hash(self)

In [5]:
def eisner(weight):
    """
    `N` denotes the length of sentence.

    :param weight: size N x N
    :return: the projective tree with maximum score
    """
    N = weight.shape[0]

    btp = {}  # Back-track pointer
    dp_s = {}

    # Init
    for i in range(N):
        for j in range(i + 1, N):
            for dir in DIRECTIONS:
                for comp in COMPLETENESS:
                    dp_s[Span(i, j, dir, comp)] = NEG_INF
    #print(dp_s)

    # base case
    for i in range(N):
        for dir in DIRECTIONS:
            dp_s[Span(i, i, dir, C)] = 0.
            btp[Span(i, i, dir, C)] = None

    rules = [
        # span_shape_tuple := (span_direction, span_completeness),
        # rule := (span_shape, (left_subspan_shape, right_subspan_shape))
        ((L, I), ((R, C), (L, C))),
        ((R, I), ((R, C), (L, C))),
        ((L, C), ((L, C), (L, I))),
        ((R, C), ((R, I), (R, C))),
    ]

    for size in range(1, N):
        for i in range(0, N - size):
            j = i + size
            for rule in rules:
                ((dir, comp), ((l_dir, l_comp), (r_dir, r_comp))) = rule

                if comp == I:
                    edge_w = weight[i, j] if (dir == R) else weight[j, i]
                    k_start, k_end = (i, j)
                    offset = 1
                else:
                    edge_w = 0.
                    k_start, k_end = (i + 1, j + 1) if dir == R else (i, j)
                    offset = 0

                span = Span(i, j, dir, comp)
                for k in range(k_start, k_end):
                    l_span = Span(i, k, l_dir, l_comp)
                    r_span = Span(k + offset, j, r_dir, r_comp)
                    s = edge_w + dp_s[l_span] + dp_s[r_span]
                    if s > dp_s[span]:
                        dp_s[span] = s
                        btp[span] = (l_span, r_span)

    # recover tree
    return back_track(btp, Span(0, N - 1, R, C), {})


def back_track(btp, span, edge_set):
    if span.complete == I:
        if span.head_side == L:
            edge = (span.right_idx, span.left_idx)
        else:
            edge = (span.left_idx, span.right_idx)
        edge_set[str(edge[1])] = str(edge[0])

    if btp[span] is not None:
        l_span, r_span = btp[span]

        back_track(btp, l_span, edge_set)
        back_track(btp, r_span, edge_set)
    else:
        return

    return edge_set

In [6]:
eisner(test_matrix)

{'2': '0', '1': '2', '3': '2'}

In [7]:
def eisner_2(scores):
    """Parse using Eisner's algorithm.
    The matrix follows the following convention:
        scores[i][j] = p(i=head, j=dep) = p(i --> j)
    """
    rows, collumns = scores.shape
    assert rows == collumns, 'scores matrix must be square'

    num_words = rows - 1  # Number of words (excluding root).

    # Initialize CKY table.
    complete = np.zeros([num_words+1, num_words+1, 2])  # s, t, direction (right=1).
    incomplete = np.zeros([num_words+1, num_words+1, 2])  # s, t, direction (right=1).
    complete_backtrack = -np.ones([num_words+1, num_words+1, 2], dtype=int)  # s, t, direction (right=1).
    incomplete_backtrack = -np.ones([num_words+1, num_words+1, 2], dtype=int)  # s, t, direction (right=1).

    incomplete[0, :, 0] -= np.inf

    # Loop from smaller items to larger items.
    for k in range(1, num_words+1):
        for s in range(num_words-k+1):
            t = s + k

            # First, create incomplete items.
            # left tree
            incomplete_vals0 = complete[s, s:t, 1] + complete[(s+1):(t+1), t, 0] + scores[t, s]
            incomplete[s, t, 0] = np.max(incomplete_vals0)
            incomplete_backtrack[s, t, 0] = s + np.argmax(incomplete_vals0)
            # right tree
            incomplete_vals1 = complete[s, s:t, 1] + complete[(s+1):(t+1), t, 0] + scores[s, t]
            incomplete[s, t, 1] = np.max(incomplete_vals1)
            incomplete_backtrack[s, t, 1] = s + np.argmax(incomplete_vals1)

            # Second, create complete items.
            # left tree
            complete_vals0 = complete[s, s:t, 0] + incomplete[s:t, t, 0]
            complete[s, t, 0] = np.max(complete_vals0)
            complete_backtrack[s, t, 0] = s + np.argmax(complete_vals0)
            # right tree
            complete_vals1 = incomplete[s, (s+1):(t+1), 1] + complete[(s+1):(t+1), t, 1]
            complete[s, t, 1] = np.max(complete_vals1)
            complete_backtrack[s, t, 1] = s + 1 + np.argmax(complete_vals1)

    value = complete[0][num_words][1]
    heads = -np.ones(num_words + 1, dtype=int)
    backtrack_eisner(incomplete_backtrack, complete_backtrack, 0, num_words, 1, 1, heads)

    value_proj = 0.0
    for m in range(1, num_words+1):
        h = heads[m]
        value_proj += scores[h, m]

    return heads


def backtrack_eisner(incomplete_backtrack, complete_backtrack, s, t, direction, complete, heads):
    """
    Backtracking step in Eisner's algorithm.
    - incomplete_backtrack is a (NW+1)-by-(NW+1) numpy array indexed by a start position,
    an end position, and a direction flag (0 means left, 1 means right). This array contains
    the arg-maxes of each step in the Eisner algorithm when building *incomplete* spans.
    - complete_backtrack is a (NW+1)-by-(NW+1) numpy array indexed by a start position,
    an end position, and a direction flag (0 means left, 1 means right). This array contains
    the arg-maxes of each step in the Eisner algorithm when building *complete* spans.
    - s is the current start of the span
    - t is the current end of the span
    - direction is 0 (left attachment) or 1 (right attachment)
    - complete is 1 if the current span is complete, and 0 otherwise
    - heads is a (NW+1)-sized numpy array of integers which is a placeholder for storing the
    head of each word.
    """
    if s == t:
        return
    if complete:
        r = complete_backtrack[s][t][direction]
        if direction == 0:
            backtrack_eisner(incomplete_backtrack, complete_backtrack, s, r, 0, 1, heads)
            backtrack_eisner(incomplete_backtrack, complete_backtrack, r, t, 0, 0, heads)
            return
        else:
            backtrack_eisner(incomplete_backtrack, complete_backtrack, s, r, 1, 0, heads)
            backtrack_eisner(incomplete_backtrack, complete_backtrack, r, t, 1, 1, heads)
            return
    else:
        r = incomplete_backtrack[s][t][direction]
        if direction == 0:
            heads[s] = t
            backtrack_eisner(incomplete_backtrack, complete_backtrack, s, r, 1, 1, heads)
            backtrack_eisner(incomplete_backtrack, complete_backtrack, r+1, t, 0, 1, heads)
            return
        else:
            heads[t] = s
            backtrack_eisner(incomplete_backtrack, complete_backtrack, s, r, 1, 1, heads)
            backtrack_eisner(incomplete_backtrack, complete_backtrack, r+1, t, 0, 1, heads)
            return

In [8]:
eisner_2(test_matrix)

array([-1,  2,  0,  2])

In [9]:
test_list = []
test_list.append((2, 1))
test_list

[(2, 1)]

In [10]:
np.zeros([4, 4])[1, 1]

0.0

In [11]:
test_matrix

array([[-10000,      9,     10,      9],
       [-10000, -10000,     20,      3],
       [-10000,     30, -10000,     30],
       [-10000,     11,      0, -10000]])

In [12]:
test_s = 1
test_t = 4

In [13]:
np.arange(1, abs(test_s-test_t)-1)

array([1])

In [14]:
test_matrix[test_s+1:test_t+1]

array([[-10000,     30, -10000,     30],
       [-10000,     11,      0, -10000]])

In [15]:
test_tree = np.zeros(4, dtype=int)
test_tree.shape[0]

4

In [16]:
"".join(["L", "_", "Closed"])

'L_Closed'

In [17]:
np.arange(test_s, test_t+1)

array([1, 2, 3, 4])

In [18]:
tree = -np.ones(4, dtype=int)

def Eisner(edge_scores):
    n=edge_scores.shape[0]
    
    # Initialize matrices with zeros
    O_r = np.zeros([n, n], dtype=np.float32)
    O_l = np.zeros([n, n], dtype=np.float32)
    C_r = np.zeros([n, n], dtype=np.float32)
    C_l = np.zeros([n, n], dtype=np.float32)
    
    # Initialize backtracking matrices
    #b_Or = -np.ones([n, n], dtype=int)
    #b_Ol = -np.ones([n, n], dtype=int)
    #b_Cr = -np.ones([n, n], dtype=int)
    #b_Cl = -np.ones([n, n], dtype=int)
    
    b_O = -np.ones([n, n, 2], dtype=int)
    b_C = -np.ones([n, n, 2], dtype=int)
    
    for m in np.arange(1, n):
        for s in np.arange(0, n-m):
            t = s+m
            
            # O_r
            O_r_list = np.array([C_l[s][q] + C_r[q+1][t] + edge_scores[t][s] for q in np.arange(s, t)])
            O_r[s][t] = np.max(O_r_list)
            
            # O_l
            O_l_list = np.array([C_l[s][q] + C_r[q+1][t] + edge_scores[s][t] for q in np.arange(s, t)])
            O_l[s][t] = np.max(O_l_list)
            
            # C_r
            C_r_list = np.array([C_r[s][q] + O_r[q][t] for q in np.arange(s, t)])
            #print(C_r_list)
            C_r[s][t] = np.max(C_r_list)
                
            # C_l
            C_l_list = np.array([O_l[s][q] + C_l[q][t] for q in np.arange(s+1, t+1)])
            #print(C_l_list)
            C_l[s][t] = np.max(C_l_list)
            
            b_O[s][t][1] = s + np.argmax(O_l_list)
            b_O[s][t][0] = s + np.argmax(O_r_list)
            b_C[s][t][0] = s + np.argmax(C_l_list)
            b_C[s][t][1] = s + 1 + np.argmax(C_r_list)
            
            #b_Or[s][t] = s + np.argmax(O_r_list)
            #b_Ol[s][t] = s + np.argmax(O_l_list)
            #b_Cl[s][t] = s + np.argmax(C_r_list)
            #b_Cr[s][t] = s + 1 + np.argmax(C_l_list)
            
            #print(np.argmax(C_l_list))
            #print(np.argmax(C_r_list))
            #print("")

    print(O_r)
    print(O_l)
    print(C_r)
    print(C_l)
    print("")
    #print(b_Or)
    #print(b_Ol)
    #print(b_Cr)
    #print(b_Cl)
    print(b_O)
    print(b_C)
    
    backtrack_dict = {"Open_R": b_Or, "Open_L": b_Ol, "Closed_R":b_Cr, "Closed_L":b_Cl}
    backtrack(0, n-1, "Closed", "R", tree, backtrack_dict)
    arcs = []
    
    #print(tree)
    
    for dep in np.arange(len(tree)):
        if dep != 0:    # Skip root
            head = tree[dep]
            arcs.append((tree[dep], dep))
    
    return arcs

def backtrack(s, t, structure, direction, tree, backtrack_dict):
    if s == t:
        return
    #print(tree)
    
    #print("".join([structure, "_", direction]))
    #print(b_table, s, t)
    #print(backpointer)
    
    if structure == "Closed":
        b_table = backtrack_dict["".join([structure, "_", direction])]
        print(b_table[s])
        backpointer = b_table[s][t]
        print(backpointer, structure, direction)
        print(s, t)
        print("")
        if direction == "L":
            backtrack(s, backpointer, "Closed", "L", tree, backtrack_dict)
            backtrack(backpointer, s, "Open", "L", tree, backtrack_dict)
            return
        else:
            backtrack(s, backpointer, "Open", "R", tree, backtrack_dict)
            backtrack(backpointer, s, "Closed", "R", tree, backtrack_dict)
            return
    else:
        b_table = backtrack_dict["".join([structure, "_", direction])]
        print(b_table[s])
        backpointer = b_table[s][t]
        print(backpointer, structure, direction)
        print(s, t)
        print("")
        if direction == "L":
            tree[s] = t
            backtrack(s, backpointer, "Closed", "R", tree, backtrack_dict)
            backtrack(backpointer+1, t, "Closed", "L", tree, backtrack_dict)
            return
        else:
            tree[t] = s
            backtrack(s, backpointer, "Closed", "R", tree, backtrack_dict)
            backtrack(backpointer+1, t, "Closed", "L", tree, backtrack_dict)
            return

In [19]:
Eisner(test_matrix)

[[     0. -10000.  -9970.  -9960.]
 [     0.      0.     30.     31.]
 [     0.      0.      0.      0.]
 [     0.      0.      0.      0.]]
[[ 0.  9. 40. 49.]
 [ 0.  0. 20. 23.]
 [ 0.  0.  0. 30.]
 [ 0.  0.  0.  0.]]
[[     0. -10000.  -9970.  -9960.]
 [     0.      0.     30.     31.]
 [     0.      0.      0.      0.]
 [     0.      0.      0.      0.]]
[[ 0.  9. 40. 70.]
 [ 0.  0. 20. 50.]
 [ 0.  0.  0. 30.]
 [ 0.  0.  0.  0.]]

[[[-1 -1]
  [ 0  0]
  [ 0  0]
  [ 2  2]]

 [[-1 -1]
  [-1 -1]
  [ 1  1]
  [ 2  2]]

 [[-1 -1]
  [-1 -1]
  [-1 -1]
  [ 2  2]]

 [[-1 -1]
  [-1 -1]
  [-1 -1]
  [-1 -1]]]
[[[-1 -1]
  [ 0  1]
  [ 1  1]
  [ 1  1]]

 [[-1 -1]
  [-1 -1]
  [ 1  2]
  [ 1  2]]

 [[-1 -1]
  [-1 -1]
  [-1 -1]
  [ 2  3]]

 [[-1 -1]
  [-1 -1]
  [-1 -1]
  [-1 -1]]]


NameError: name 'b_Or' is not defined

In [ ]:
def parse(score):
    # pass in score matrix
    # sentence includes  ROOT
    N = score.shape[0]

    # initialize the tables, third dim represents the head side
    o = np.zeros([N, N, 2])  # right = 1
    close = np.zeros([N, N, 2])  # right = 1

    # initialize back pointer tables
    open_bt = -np.ones([N, N, 2], dtype=int)  # right = 1
    close_bt = -np.ones([N, N, 2], dtype=int)  # right = 1

    # fill in the score of arcs
    for m in range(1, N):
        for s in range(0, N-m):
            t = s+m
            # open tables
            # left head
            open_val0 = close[s, s:t, 1] + \
                close[(s+1):(t+1), t, 0] + score[t, s]
            o[s, t, 0] = np.max(open_val0)
            open_bt[s][t][0] = s + np.argmax(open_val0)
            # right head
            open_val1 = close[s, s:t, 1] + \
                close[(s+1):(t+1), t, 0] + score[s, t]
            o[s, t, 1] = np.max(open_val1)
            open_bt[s][t][1] = s + np.argmax(open_val1)

            # closed tables
            # left head
            close_val0 = close[s, s:t, 0] + o[s:t, t, 0]
            close[s, t, 0] = np.max(close_val0)
            close_bt[s, t, 0] = s + np.argmax(close_val0)
            # right head
            close_val1 = o[s, (s+1):(t+1), 1] + close[(s+1):(t+1), t, 1]
            close[s, t, 1] = np.max(close_val1)
            close_bt[s, t, 1] = s + 1 + np.argmax(close_val1)
            
            #print(np.argmax(close_val1))
            #print(np.argmax(close_val0))
            #print("")

            
    #print(o[:, :, 1])
    #print(o[:, :, 0])
    #print(close[:, :, 1])
    #print(close[:, :, 0])
    #print("")
    #print(open_bt[:, :, 1])
    #print(open_bt[:, :, 0])
    #print(close_bt[:, :, 1])
    #print(close_bt[:, :, 0])
    #print(open_bt)
    #print(close_bt)
            
    # initial a list to represent the tree
    tree = -np.ones(N, dtype=int)
    backtrack(open_bt, close_bt, 0, N-1, 1, 1, tree)

    edge_set = {}
    for i, head in enumerate(tree):
        if i == 0:
            continue
        else:
            edge_set[str(i)] = str(head)

    return edge_set

def backtrack(open_bt, close_bt, s, t, direction, complete, tree):

    if s == t:
        return
    
    #print(close_bt)
    #print(close_bt[s][t])
    
    if complete:
        #print(close_bt[s][:])
        r = close_bt[s][t][direction]
        #print(r, complete, direction)
        #print(s, t)
        #print("")
        if direction == 0:
            backtrack(open_bt, close_bt, s, r, 0, 1, tree)
            backtrack(open_bt, close_bt, r, t, 0, 0, tree)
            return
        else:
            backtrack(open_bt, close_bt, s, r, 1, 0, tree)
            backtrack(open_bt, close_bt, r, t, 1, 1, tree)
            return
    else:
        #print(open_bt[s][:])
        r = open_bt[s][t][direction]
        #print(r, complete, direction)
        #print(s, t)
        #print("")
        if direction == 0:
            tree[s] = t
            backtrack(open_bt, close_bt, s, r, 1, 1, tree)
            backtrack(open_bt, close_bt, r+1, t, 0, 1, tree)
            return
        else:
            tree[t] = s
            backtrack(open_bt, close_bt, s, r, 1, 1, tree)
            backtrack(open_bt, close_bt, r+1, t, 0, 1, tree)
            return

In [20]:
parse(test_matrix)

NameError: name 'parse' is not defined

In [21]:
Eisner(test_matrix)

[[     0. -10000.  -9970.  -9960.]
 [     0.      0.     30.     31.]
 [     0.      0.      0.      0.]
 [     0.      0.      0.      0.]]
[[ 0.  9. 40. 49.]
 [ 0.  0. 20. 23.]
 [ 0.  0.  0. 30.]
 [ 0.  0.  0.  0.]]
[[     0. -10000.  -9970.  -9960.]
 [     0.      0.     30.     31.]
 [     0.      0.      0.      0.]
 [     0.      0.      0.      0.]]
[[ 0.  9. 40. 70.]
 [ 0.  0. 20. 50.]
 [ 0.  0.  0. 30.]
 [ 0.  0.  0.  0.]]

[[[-1 -1]
  [ 0  0]
  [ 0  0]
  [ 2  2]]

 [[-1 -1]
  [-1 -1]
  [ 1  1]
  [ 2  2]]

 [[-1 -1]
  [-1 -1]
  [-1 -1]
  [ 2  2]]

 [[-1 -1]
  [-1 -1]
  [-1 -1]
  [-1 -1]]]
[[[-1 -1]
  [ 0  1]
  [ 1  1]
  [ 1  1]]

 [[-1 -1]
  [-1 -1]
  [ 1  2]
  [ 1  2]]

 [[-1 -1]
  [-1 -1]
  [-1 -1]
  [ 2  3]]

 [[-1 -1]
  [-1 -1]
  [-1 -1]
  [-1 -1]]]


NameError: name 'b_Or' is not defined

# Main

In [22]:
train_path_en = "Treebanks/english/train/wsj_train.only-projective.conll06"
reader = Reader(train_path_en)
test_data = Data(reader.read_file())
test_data.sentences

 ...]

In [23]:
train_path_de = "Treebanks/german/train/tiger-2.2.train.only-projective.first-1k.conll06"
reader = Reader(train_path_de)
test_data = Data(reader.read_file())
test_data.sentences

In [24]:
test_mapping = FeatureMapping(test_data.sentences)
test_mapping.create_map()

1000 sentences mapped


In [25]:
len(test_mapping.map)

2824143

In [26]:
test_data.sentences[0].tokens[2].form

'Ross'

In [27]:
test_model = Model(test_data, test_mapping)

In [29]:
test_model.train(3)

Start time: 65.116067625
Epoch: 1
Time taken for past 100 sentences: 1.0634974999999969
Time taken for past 200 sentences: 2.237661375000002
Time taken for past 300 sentences: 3.240449416000004
Time taken for past 400 sentences: 3.993228415999994
Time taken for past 500 sentences: 5.028140500000006
Time taken for past 600 sentences: 5.959402791000002
Time taken for past 700 sentences: 6.7791041250000035
Time taken for past 800 sentences: 7.524140583000005
Time taken for past 900 sentences: 8.505213208
Time taken for past 1000 sentences: 9.589795166000002
UAS score on 17166 tokens over 1000 sentences: 0.6811720843527904
Time taken for epoch: 9.593091666000007
0.0

Start time: 74.90859175
Epoch: 2
Time taken for past 100 sentences: 0.8634557910000069
Time taken for past 200 sentences: 1.9233498330000032
Time taken for past 300 sentences: 2.8459392910000076
Time taken for past 400 sentences: 3.760132749999997
Time taken for past 500 sentences: 4.7125152910000025
Time taken for past 600 se

In [30]:
writer = Writer(train_path_de.split('/')[-1], test_data.sentences)
writer.write_file()

In [ ]:
from difflib import Differ
 
with open(train_path_en) as file_1, open(train_path_en.split('/')[-1]+'.pred') as file_2:
    differ = Differ()
 
    for line in differ.compare(file_1.readlines(), file_2.readlines()):
        print(line)